In [ ]:
pip instal python-telegram-bot

In [ ]:
import telegram

token = ""

bot = telegram.Bot(token = token) # Bot 선언

updates = bot.getUpdates() # 업데이트 내역
 
for u in updates: # 내역중 메세지 출력
    print(u.message)

In [ ]:
#chat_id = updates[-1].message.chat.id #가장 최근에 온 메세지의 chat id를 가져옵니다
chat_id = ""
chat_id

In [ ]:
bot.sendMessage(chat_id = chat_id, text = "안녕하세요.")

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req
from datetime import datetime as dt

url = "http://www.naver.com"
res = req.urlopen(url)

data = BeautifulSoup(res, "html.parser")

rank = data.select(".ah_r")
word = data.select(".ah_k")

message = "[네이버 실시간 검색어 순위]\n" 
message += str(dt.now()) + '\n'
for r, w in zip(rank[:20], word[:20]):
    message += r.string + ' ' + w.string + '\n'
print(message)


#bot.sendMessage(chat_id = chat_id, text = message)
bot.sendMessage(chat_id = "", text=message)

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req
from datetime import datetime as dt
import time

while True:
    url = "http://www.naver.com"
    res = req.urlopen(url)

    data = BeautifulSoup(res, "html.parser")

    rank = data.select(".ah_r")
    word = data.select(".ah_k")

    message = "[네이버 실시간 검색어 순위]\n" 
    message += str(dt.now()) + '\n'
    for r, w in zip(rank[:20], word[:20]):
        message += r.string + ' ' + w.string + '\n'
    print(message)


    bot.sendMessage(chat_id = chat_id, text = message)
    
    time.sleep(10)

In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req
from urllib import parse
from datetime import datetime as dt

keyword = "대구"
url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&"
query = [ ('query', keyword)]
query = parse.urlencode(query, encoding='UTF-8', doseq=True)
url = url + query

res = req.urlopen(url)

data = BeautifulSoup(res, "html.parser")

tag = data.select("._sp_each_title")

message = "[네이버 {} 뉴스]\n".format(keyword) 
message += str(dt.now()) + '\n'
for a in tag:
    message +=  a['title'] +a['href'] + '\n'
message

In [ ]:
bot.sendMessage(chat_id = chat_id, text = message)


여기서 CommandHandler 는 /keyword 메세지 를 들어 올때는 의미

/ 가 붙으면 커맨더 명령이라고 인식,  dispatcher에 해당 핸들러를 추가 

/keyword 채팅이 왔을때 입력한 keyword로 뉴스 검색해서 결과값 반환

In [ ]:
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler  


def getdata(kw):
    from bs4 import BeautifulSoup
    import urllib.request as req
    from urllib import parse
    from datetime import datetime as dt

    keyword = kw
    url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&"
    query = [ ('query', keyword)]
    query = parse.urlencode(query, encoding='UTF-8', doseq=True)
    url = url + query

    res = req.urlopen(url)

    data = BeautifulSoup(res, "html.parser")

    tag = data.select("._sp_each_title")

    message = "[네이버 {} 뉴스]\n".format(keyword) 
    message += str(dt.now()) + '\n'
    for a in tag:
        message +=  a['title'] +a['href'] + '\n'
    return message


# help reply function
def keyword_command(bot, update) :
    kw = update.message.text.split(' ')[-1]
    message = getdata(kw)
    update.message.reply_text(message)


updater = Updater(token)

keyword_handler = CommandHandler('keyword', keyword_command)
updater.dispatcher.add_handler(keyword_handler)

updater.start_polling(timeout=3, clean=True)
updater.idle()


위의 코드가 계속해서 실행되어야만 이벤트를 탐지하고 메세지를 보낼 수 있음.

사용자가 해당 봇으로 /keyword 검색어 를 입력하면

검색어만 추출하여 kw변수에 저장 후 해당 검색어 뉴스를 조회해서 결과로 전송